In [11]:
from faker import Faker
import random
import hashlib
import datetime
# import json
# fake = Faker('ru_RU')
fake = Faker()

In [12]:
class Profile:
    def __init__(self,id,time):
        self.id = id
        self.gender = random.choice(('M', 'F'))
        self.name = fake.name_male() if self.gender == 'M' else fake.name_female()
        self.birthday = fake.date_of_birth(minimum_age=12, maximum_age=80).isoformat()
        self.email = fake.email()
        self.phone = fake.msisdn()
        self.salt = fake.password(length=8)
        self.hash = hashlib.sha1((fake.password() + self.salt).encode('utf-8')).hexdigest()
        self.created = time  # fake.past_datetime().isoformat()
        self.intro = fake.paragraph(nb_sentences=3)
        avatarStr = f'https://eu.ui-avatars.com/api/?name={self.name}&background=random&size={{}}'
        self.avatarS = avatarStr.format(32)
        self.avatarL = avatarStr.format(128)
        self.nextPost = 0
        self.nextNotif = 0
        self.nextPhoto = 0
        
    def __str__(self):
        return (f'{self.id},{self.name},{self.gender},{self.birthday},{self.email},{self.phone},'
                f'{self.salt},{self.hash},{self.created},{self.intro},{self.avatarS},{self.avatarL},'
                f'{self.nextPost},{self.nextNotif},{self.nextPhoto}\n')

class Post:
    def __init__(self,id,userId,time,atchType='',atchId=''):
        self.id = id
        self.userId = userId
        self.created = time # date_time_between(start_date=SINCE_CREATED)
#         self.modified = fake.date_between_dates(date_start=self.created) if not random.randint(0,10) else None
        self.tags = '-'.join(fake.words(nb=random.randint(0,3)))
        self.content = fake.paragraph(nb_sentences=8)
        self.atchType = atchType
        self.atchId = atchId
        self.reaction = [0,0,0,0,0,0]
#         self.nextReact = 1
        self.nextCmt = random.randint(0,50)
        
    def __str__(self):
        return (f'{self.id},{self.userId},{self.created},,{self.tags},{self.content},{self.atchType},'
                f'{self.atchId},,"{{{",".join(str(r) for r in self.reaction)}}}",'
                f'{sum(self.reaction)},{self.nextCmt}\n').replace(',None,',',,').replace(',None\n',',\n')

class Comment:
    def __init__(self,id,userId,postId,created,parentId=0,replyCount=0):
        self.id = id
        self.userId = userId
        self.postId = postId
        self.parentId = parentId
        self.content = fake.paragraph(nb_sentences=2)
        self.created = created
        self.replyCount = replyCount
        
    def __str__(self):
        return (f'{self.id},{self.userId},{self.postId},{self.parentId},{self.content},'
                f'{self.created},{self.replyCount}\n').replace(',None,',',,').replace(',None\n',',\n')

class Reaction:
    def __init__(self,userId,postId,t):
        self.userId = userId
        self.postId = postId
        self.type = t
    def __str__(self):
        return (f'{self.userId},{self.postId},{self.type}\n')

class Relationship:
    def __init__(self,user1,user2,t,created):
        self.user1 = user1
        self.user2 = user2
        self.created = created
        self.type = t
    def __str__(self):
        return (f'{self.user1},{self.user2},{self.created},{self.type},\n')

class Notification:
    def __init__(self,id,userId,t,created,fromUserId,postId='',cmtId=''):
        self.id = id
        self.userId = userId
        self.type = t
        self.created = created
        self.fromUserId = fromUserId
        self.postId = postId
        self.cmtId = cmtId
    def __str__(self):
        return (f'{self.id},{self.userId},{self.type},{self.created},{self.fromUserId},{self.postId},{self.cmtId}\n').replace(',None,',',,').replace(',None\n',',\n')

class Album:
    def __init__(self,id,userId,descr,created):
        self.id = id
        self.userId = userId
        self.descr = descr
        self.created = created
    def __str__(self):
        return (f'{self.id},{self.userId},{self.descr},{self.created}\n')

class Photo:
    def __init__(self,id,userId,created,albumId=1):
        self.id = id
        self.userId = userId
        self.albumId = albumId
        self.url = 'https://loremflickr.com/320/240/all'
        self.created = created
    def __str__(self):
        return (f'{self.id},{self.userId},{self.albumId},{self.url},{self.created}\n')

In [13]:
SINCE_CREATED = '-10y'

class User:
    def __init__(self, id, time):
        time = datetime.datetime.fromtimestamp(time).isoformat()
        self.id = id
        self.profile = Profile(id, time)
        self.posts = []
        self.comments = []
        self.reactions = {}
        self.relationships = []
        self.notifications = []
        self.albums = [Album(0,id,'Avatar',time),Album(1,id,'Upload',time)]
        self.photos = [Photo(0, id, time, 0)]
        self.profile.nextPhoto += 1
        
    def send_friend_request(self, user, time):
        if self.id == user.id:
            return
        time = datetime.datetime.fromtimestamp(time).isoformat()
        user.relationships.append(Relationship(self.id, user.id, 'request', time))
        user.notifications.append(Notification(user.profile.nextNotif, user.id, 'request', time, self.id))
        user.profile.nextNotif += 1
        
    def accept_friend_request(self, user, time):
        time = datetime.datetime.fromtimestamp(time).isoformat()
        for r in self.relationships:
            if user.id == r.user1 and r.type == 'request':
                r.type == 'friend'
                r.time = time
                user.relationships.append(Relationship(user.id, self.id, 'friend', time))
                user.notifications.append(Notification(user.profile.nextNotif, user.id, 'accept', time, self.id))
                user.profile.nextNotif += 1
                return
        self.relationships.append(Relationship(self.id, user.id, 'friend', time))
        self.notifications.append(Notification(self.profile.nextNotif, self.id, 'request', time, user.id))
        self.profile.nextNotif += 1
        user.relationships.append(Relationship(user.id, self.id, 'friend', time))
        user.notifications.append(Notification(user.profile.nextNotif, user.id, 'accept', time, self.id))
        user.profile.nextNotif += 1
                
    def post(self, time):
        time = datetime.datetime.fromtimestamp(time).isoformat()
        if not random.randint(0,3):
            self.photos.append(Photo(self.profile.nextPhoto, self.id, time))
            self.posts.append(Post(self.profile.nextPost, self.id, time, 'photo', self.profile.nextPost))
            self.profile.nextPhoto += 1
        else:
            self.posts.append(Post(self.profile.nextPost, self.id, time))
        self.profile.nextPost += 1
        
    def react(self, user):
        if user.profile.nextPost == 0:
            return
        postId = random.randint(0,user.profile.nextPost-1)
        key = f'{self.id},{user.id},{postId}'
        if not self.reactions.get(key):
            c = random.choices((0,1,2,3,4,5), (5,2,1,1,1,1), k=1)[0]
            self.reactions[key] = ('like','love','haha','wow','sad','angry')[c]
            user.posts[postId].reaction[c] += 1
#             user.posts[postId].nextReact += 1
            
    def comment(self, user, time):
        if user.profile.nextPost == 0:
            return
        time = datetime.datetime.fromtimestamp(time).isoformat()
        postId = random.randint(0,user.profile.nextPost-1)
        if not random.randint(0,2):
            parentId = random.randint(-1,user.posts[postId].nextCmt-1)
#             if parentId >= 0:
#                 self.comments[parentId].replyCount += 1
            self.comments.append(Comment(user.posts[postId].nextCmt, user.id, postId, time, parentId))
        else:
            self.comments.append(Comment(user.posts[postId].nextCmt, user.id, postId, time))
            user.notifications.append(Notification(user.profile.nextNotif, user.id, 'cmt', time, self.id, postId, user.posts[postId].nextCmt))
            user.profile.nextNotif += 1

        user.posts[postId].nextCmt += 1

In [14]:
class SocialNetwork:
    def generate(self, dt=100000, n=100):
        now = int(datetime.datetime.now().timestamp())
        t = now - dt
        users = []
        
        i = 0
        
        while i < n:
            t += random.randint(1,1000)
            users.append(User(i,t))
            i += 1
            
            if random.randint(1,3) == 1:
                random.choice(users).post(t)

        while t < now:
            t += random.randint(1,80)
            c = random.randint(1,1500)
            
            if c == 1:
                users.append(User(i,t))
                i += 1
            elif c < 10:
                u = random.choice(users)
                random.choice(users).send_friend_request(u, t)
            elif c < 30:
                u = random.choice(users)
                random.choice(users).accept_friend_request(u, t)
            elif c < 100:
                random.choice(users).post(t)
            elif c < 500:
                u = random.choice(users)
                random.choice(users).comment(u, t)
            else:
                u = random.choice(users)
                random.choice(users).react(u)
        
        self.users = users
        print(len(users))

                
    def save(self, save_dir='csv/'):
        f1 = open(save_dir+'profile.csv', 'w')
        f2 = open(save_dir+'post.csv', 'w')
        f3 = open(save_dir+'comment.csv', 'w')
        f4 = open(save_dir+'reaction.csv', 'w')
        f5 = open(save_dir+'relationship.csv', 'w')
        f6 = open(save_dir+'notification.csv', 'w')
        f7 = open(save_dir+'album.csv', 'w')
        f8 = open(save_dir+'photo.csv', 'w')

        f1.write('Id,Name,Gender,Birthday,Email,Phone,Salt,Hash,Created,Intro,AvatarS,AvatarL,Nextost,NextNotif,NextPhoto\n')
        f2.write('Id,UserId,Created,Modified,Tags,Content,AtchType,AtchId,AtchUrl,Reaction,NextCmt\n')
        f3.write('Id,UserId,PostId,ParentId,Content,Created,ReplyCount\n')
        f4.write('UserId,AuthorId,PostId,Type\n')
        f5.write('User1,User2,Created,Type,Other\n')
        f6.write('Id,UserId,Type,Created,FromUserId,PostId,CmtId\n')
        f7.write('Id,UserId,Description,Created\n')
        f8.write('Id,UserId,AlbumId,Url,Created\n')
        
        for user in self.users:
            f1.write(str(user.profile))
            f2.write(''.join(str(o) for o in user.posts))
            f3.write(''.join(str(o) for o in user.comments))
            for k, v in user.reactions.items():
                f4.write(f'{k},{v}\n')
            f5.write(''.join(str(o) for o in user.relationships))
            f6.write(''.join(str(o) for o in user.notifications))
            f7.write(''.join(str(o) for o in user.albums))
            f8.write(''.join(str(o) for o in user.photos))
        
        f1.close()
        f2.close()
        f3.close()
        f4.close()
        f5.close()
        f6.close()
        f7.close()
        f8.close()

In [15]:
%%time
Net = SocialNetwork()
Net.generate(100000000)
Net.save()

1735
CPU times: user 2min 48s, sys: 854 ms, total: 2min 49s
Wall time: 2min 50s


In [6]:
class Coeff:
    def __init(self):
        self.friend = random.random()
        self.post = random.random()
        self.comment = random.random()
        self.reaction = random.random()
        self.popularity = random.random()
        print(self.friend,self.post,self.comment,self.reaction,self.popularity)

2000 user
1 user 20 post
1 user 50 notification
1 user 200 relationship
1 post 20 comment
1 post 100 reaction